### Cpahter 13. 연산자 오버로딩

파이썬에서는 +(`__add__`) 나 ==(`__eq__`) 등 다양한 연산자를 오버로딩할 수 있다. is, and, or, not 연산자는 오버로딩할 수 없지만, 그 외에 우리가 볼 수 있는 대부분의 연산자, 심지어 슬라이싱([])까지도 오버로딩할 수 있다. 이 단원에서는 다양한 연산자들의 오버로딩중 일부에 대해 알아본다.

먼저 단항 연산자 오버로딩에 대해서 알아본다. 내장 자료형에 대한 연산자는 오버로딩할 수 없으므로, 오버로딩 공부를 위해 리스트를 안에 넣은 간단한 클래스를 만들자. 그리고 3가지의 단항 연산자에 대한 오버로딩 코드를 작성한다.

In [3]:
import math

class study:
  def __init__(self, members):
    self.nums=list(members)

  def __str__(self):
    return str(self.nums)
  
  def __abs__(self): # 절댓값 연산자 오버로딩
    return math.sqrt(sum(x*x for x in self.nums))

  def __neg__(self): # 단항 산술 부정
    return study(-x for x in self.nums)

  def __pos__(self): #단항 산술 덧셈. 보통은 x와 +x는 같지만, 아주 일부의 경우 다를 때가 발생한다.
    return study(self.nums)

s=study([2,3,4,5,6])
print(+s)
print(-s)
print(abs(s))

[2, 3, 4, 5, 6]
[-2, -3, -4, -5, -6]
9.486832980505138


단항 연산자는 인수를 self 하나만 받으므로 구현하기 쉽다. 단 연산자 오버로딩을 구현할 때는 지켜야 하는 핵심 규칙이 있는데, '언제나 새로운 객체를 반환해야 한다' 는 것이다. 따라서 self 와 관련된 객체를 반환해서는 안 된다. 반드시 클래스 생성자를 이용해 새로운 객체를 생성해 반환하자.

이때 단항 연산자는 하나 더 있는데, ~(`__invert__`, 비트 반전 연산자)이다. 여기서는 구현하지 않았다.

+ 덧셈 연산자 오버로딩

아까와 같은 클래스를 사용해서 간단한 덧셈을 오버로딩해 보자. 그런데 생각해야 할 것은, 만약에 더하는 두 클래스의 길이가 다르면 어떻게 할 것인가? 두 벡터를 더하는데, 길이가 다르면 그 두 개를 더한 결과는 어떻게 만들어야 하는가? 에러를 띄울 수도 있겠지만, 길이가 긴 벡터의 길이에 맞추고, 짧은 벡터의 나머지 요소는 0으로 채워져 있다고 생각하는 게 낫다. 코드는 다음과 같다.


In [4]:
import itertools

class study:
  def __init__(self, members):
    self.nums=list(members)

  def __str__(self):
    return str(self.nums)

  def __add__(self,other):
    pairs=itertools.zip_longest(self.nums,other.nums,fillvalue=0)
    #zip_longest 함수는 어떤 반복형 객체라도 사용할 수 있다.
    return study(a+b for a,b, in pairs)

s1=study([2,3,4,5,6])
s2=study([10,20,30,40,50])
print(s1+s2)

[12, 23, 34, 45, 56]


+ 곱셈 연산자 오버로딩

리스트를 스칼라와 곱한다고 해보자. 가령 [1,2,3]*10은 [10,20,30] 이 되는 것이다. 그런데 이때 어떤 값이 스칼라인지 판별할 수 있어야 한다. 이를 위해서는 적당한 추상 베이스 클래스를 이용해서 `isinstance` 로 검사한다.



In [6]:
import numbers

class study:
  def __init__(self, members):
    self.nums=list(members)

  def __str__(self):
    return str(self.nums)

  def __mul__(self,scalar):
    if isinstance(scalar, numbers.Real): # 이렇게 추상 베이스 클래스를 활용하면 단순한 정수형, 실수형 뿐 아니라 Fraction 등 수치형의 값으로도 곱할 수 있다.
      return study(n*scalar for n in self.nums)
    else: return NotImplemented
  
  def __rmul__(self,scalar):
    return self*scalar

s1=study([1,2,3,4,5])
print(3*s1)

[3, 6, 9, 12, 15]


+ 비교 연산자

비교 연산자 == 와 !=는 만약 비교가 실패할 시, 역순(즉, 원래 연산이 `a.__eq__(b)` 였다면 `b.__eq__(a)` 를 시도하는 것)을 시도하고, 최후의 수단으로는 두 객체의 id를 비교한다.

+ 복합 할당 연산자 오버로딩

파이썬의 복합 할당 연산자는 가변형 객체에 적용될 경우, 새로운 객체를 만들어 주는 것이 아니라 원래의 객체를 변경한다. 다음 코드를 보면 알 수 있다.



In [7]:
a=[1,2,3]
b=[4,5]
print("a :",a,"id of a:",id(a))
a+=b
print("a :",a,"id of a:",id(a))

a : [1, 2, 3] id of a: 140488336501576
a : [1, 2, 3, 4, 5] id of a: 140488336501576


위 코드에서 `a+=b`를 실행하자 a는 바뀌었지만 id(a)는 그대로이다. 새로운 객체를 만들어서 a에 할당해 준 게 아니라 원래 a가 가리키고 있던 객체를 변경해 준 것이다. 따라서 복합 할당 연산자 오버로딩 시에도 그런 사항을 고려해 주는 것이 좋다. 물론 `__add__` 만 오버로딩한다면 자동으로 덧셈 복합 할당 연산자도 작동한다. `a+=b` 가 `a=a+b` 와 같이 취급되기 때문이다. 복합 할당 연산자를 따로 구현하기 위해서는 `__iadd__` 메서드를 구현해야 한다.



In [10]:
import itertools

class study:
  def __init__(self, members):
    self.nums=list(members)

  def __str__(self):
    return str(self.nums)

  def __add__(self,other):
    pairs=itertools.zip_longest(self.nums,other.nums,fillvalue=0)
    #zip_longest 함수는 어떤 반복형 객체라도 사용할 수 있다.
    return study(a+b for a,b, in pairs)

  def __iadd__(self, other):
    if len(other.nums)>len(self.nums):
      left=len(other.nums)-len(self.nums)
      self.nums.extend([0 for i in range(left)])
    for i in range(len(self.nums)):
      self.nums[i]+=other.nums[i]
    return self

s1=study([2,3,4,5,6])
s2=study([10,20,30,40,50,60])
print(s1, id(s1))
s1+=s2
print(s1,id(s1))

[2, 3, 4, 5, 6] 140488335168904
[12, 23, 34, 45, 56, 60] 140488335168904


위 코드와 같이 구현하면, 약간 비효율적이긴 하지만 기존의 객체를 변경하지 않고 복합 할당 덧셈 연산자가 작동한다.